# Data Engineering for Databases

In [114]:
import pickle
from datetime import date

import boto3
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import configparser

In [76]:
with open('all_votes.p', 'rb') as f:
    votes = pickle.load(f)
    
with open('temp_senators.p', 'rb') as f:
    senators = pickle.load(f)

## Preparing data for database insertion

### Table for senate members
- SEN_ID TEXT PRIMARY key NOT NULL
- F_NAME TEXT NOT NULL
- L_NAME TEXT NOT NULL
- PARTY TEXT NOT NULL
- GENDER TEXT NOT NULL
- STATE TEXT NOT NULL

In [77]:
# Already in format for insertion into SQL database
senators[0]

{'id': 'A000031',
 'first_name': 'Brockman',
 'last_name': 'Adams',
 'party': 'D',
 'gender': None,
 'state': 'WA'}

#### Check gender self-identification
Determine senator gender self-identification to replace missing values: F - Female, M - Male, N - Non-Binary.

In [80]:
# Deal with gender NaN values
for senator in senators:
    if senator['gender'] == None:
        print(senator['id'], f'{senator["first_name"]} {senator["last_name"]}')

A000031 Brockman Adams
B000401 Lloyd Bentsen
B000647 Rudolph Boschwitz
B001077 Quentin Burdick
C000877 Alan Cranston
D000366 Alan Dixon
F000329 Wyche Fowler
H000951 Gordon Humphrey
M000250 Spark Matsunaga
P000513 Larry Pressler
S001138 Steven Symms


In [81]:
# Manual look up
gender = {'A000031': 'M',
          'B000401': 'M',
          'B000647': 'M',
          'B001077': 'M',
          'C000877': 'M',
          'D000366': 'M',
          'F000329': 'M',
          'H000951': 'M',
          'M000250': 'M',
          'P000513': 'M',
          'S001138': 'M'}

In [82]:
# Correct data
for senator in senators:
    for k, v in gender.items():
        if senator['id'] == k:
            senator['gender'] = v
        else:
            continue

In [83]:
senators[0]

{'id': 'A000031',
 'first_name': 'Brockman',
 'last_name': 'Adams',
 'party': 'D',
 'gender': 'M',
 'state': 'WA'}

### Table for bills and table for votes
#### Bills
- CSR_ID TEXT PRIMARY key NOT NULL (unique primary key will be constructed from 'congress.session.roll_call')
- CONGRESS INT NOT NULL
- SESSION INT NOT NULL
- ROLL_CALL INT NOT NULL
- DATE DATE NOT NULL

#### Votes
- ID INT PRIMARY KEY NOT NULL AUTO INCREMENT
- SEN_ID TEXT FOREIGN key NOT NULL
- CSR_ID TEXT FOREIGN key NOT NULL
- POSITION TEXT NOT NULL

In [84]:
# Some 'bills' are not bills but confirmations, treaty votes, etc. and will be dropped
votes[0]['bill_id']

'-101'

In [85]:
# Integer bill_ids are codes for special votes as mentioned above
new_votes = []
for vote in votes:
    try:
        int(vote['bill_id'])
    except:
        new_votes.append(vote)

In [86]:
print(len(votes))
print(len(new_votes))

8949
8401


In [87]:
# List of bills for bills table
dict_bills = []

# List of votes for votes table
dict_votes = []

for vote in new_votes:
    vote['csr_id'] = f'{vote["congress"]}.{vote["session"]}.{vote["roll_call"]}'
    bill = {
        'csr_id': vote['csr_id'],
        'congress': vote['congress'],
        'session': vote['session'],
        'roll_call': vote['roll_call'],
        'bill_id': vote['bill_id'],
        'date': vote['date']
    }
    dict_bills.append(bill)
    
    for position in vote['positions']:
        p = {
            'sen_id': position['member_id'],
            'csr_id': vote['csr_id'],
            'position': position['vote_position']
        }
        dict_votes.append(p)

In [88]:
dict_votes[0]

{'sen_id': 'A000031', 'csr_id': '101.1.11', 'position': 'Yes'}

In [89]:
dict_bills[0]

{'csr_id': '101.1.11',
 'congress': 101,
 'session': 1,
 'roll_call': 11,
 'bill_id': 's.j.res.7-101',
 'date': '1989-02-02'}

## Data to Database
Each table was inserted one at a time (it is far more efficient to create a list of tables to be inserted).

In [38]:
config = configparser.ConfigParser()
config.read('config.ini')
ENDPOINT = config.get('aws', 'ENDPOINT')
PORT = config.get('aws', 'PORT')
USR = config.get('aws', 'USER')
PWD = config.get('aws', 'PASSWORD')

In [39]:
# Establish connection to AWS
conn = psycopg2.connect(
    host=ENDPOINT,
    user=USR,
    password=PWD,
    port=PORT,
)

# Set connection to autocommit
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [52]:
# Instantiate cursor
cursor = conn.cursor()

In [53]:
# Create sen_db
# DO NOT RUN
# cursor.execute("CREATE DATABASE sen_db;")

In [103]:
# Connect to database
conn = psycopg2.connect(
    host=ENDPOINT,
    user=USR,
    password=PWD,
    port=PORT,
    database='sen_db',
)

conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()

### Senators Table

In [110]:
# Create senators table
# DO NOT RUN
# cursor.execute(
#     """
#     CREATE TABLE senators (
#     sen_id TEXT PRIMARY KEY NOT NULL,
#     f_name TEXT NOT NULL,
#     l_name TEXT NOT NULL,
#     party TEXT NOT NULL,
#     gender TEXT NOT NULL,
#     state TEXT NOT NULL
#     );
#     """
# )

In [106]:
# Add senator operation for cursor execution
add_senator = (
    """
    INSERT INTO senators (sen_id, f_name, l_name, party, gender, state)
    VALUES (%s, %s, %s, %s, %s, %s);
    """
)

In [107]:
# Create list of rows to insert
sen_data = []
for senator in senators:
    data = (
        senator['id'],
        senator['first_name'],
        senator['last_name'],
        senator['party'],
        senator['gender'],
        senator['state']
    )
    sen_data.append(data)

In [108]:
# Insert rows
# DO NOT RUN
# cursor.executemany(add_senator, sen_data)

In [112]:
print('Number of rows in database:', cursor.rowcount) # 292
print('Number of senators:', len(senators)) # 292

Number of rows in database: 292
Number of senators: 292


### Bills Table

In [131]:
# Create bills table
# DO NOT RUN
# cursor.execute(
#     """
#     CREATE TABLE bills (
#     csr_id TEXT PRIMARY KEY NOT NULL,
#     congress INT NOT NULL,
#     session INT NOT NULL,
#     roll_call INT NOT NULL,
#     bill_id TEXT NOT NULL,
#     date DATE NOT NULL
#     );
#     """
# )

In [136]:
# Add bill operation for cursor execution
add_bill = (
    """
    INSERT INTO bills (csr_id, congress, session, roll_call, bill_id, date)
    VALUES (%s, %s, %s, %s, %s, %s);
    """
)

In [137]:
# Create list of rows to insert
bill_data = []
for bill in dict_bills:
    ymd = [ int(x) for x in bill['date'].split('-') ]
    data = (
        bill['csr_id'],
        bill['congress'],
        bill['session'],
        bill['roll_call'],
        bill['bill_id'],
        date(ymd[0], ymd[1], ymd[2])
    )
    bill_data.append(data)

In [138]:
# Insert rows
# DO NOT RUN
# cursor.executemany(add_bill, bill_data)

In [139]:
print('Number of rows in database:', cursor.rowcount) # 8401
print('Number of bills:', len(dict_bills)) # 8401

Number of rows in database: 8401
Number of bills: 8401


### Votes Table

In [141]:
# Create votes table, id is autoincrement
# DO NOT RUN
# cursor.execute(
#     """
#     CREATE TABLE votes (
#     id SERIAL PRIMARY KEY,
#     sen_id TEXT references senators(sen_id),
#     csr_id TEXT references bills(csr_id),
#     position TEXT NOT NULL
#     );
#     """
# )

In [142]:
# Add vote operation for cursor execution
add_vote = (
    """
    INSERT INTO votes (sen_id, csr_id, position)
    VALUES (%s, %s, %s);
    """
)

In [145]:
# Create list of rows to insert
vote_data = []
for vote in dict_votes:
    data = (
        vote['sen_id'],
        vote['csr_id'],
        vote['position']
    )
    vote_data.append(data)

In [ ]:
# Insert rows
# DO NOT RUN
# cursor.executemany(add_vote, vote_data)

In [ ]:
print('Number of rows in database:', cursor.rowcount) # 
print('Number of bills:', len(dict_votes)) # 